In [1]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor

2023-12-21 18:05:11.052724: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 18:05:11.214307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 18:05:11.214345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 18:05:11.219599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 18:05:11.243452: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 18:05:11.244588: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
preprocessor = create_preprocessor("resnet50", target_size=(224, 224))

In [4]:
path = "../illustrations/guess-me.jpg"
x = preprocessor.from_path(path)

In [5]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [8]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "cats-classifier"
pb_request.model_spec.signature_name = "serving_default"

pb_request.inputs["input_2"].CopyFrom(np_to_protobuf(x))

In [9]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [10]:
pb_response

model_spec {
  name: "cats-classifier"
  version {
    value: 1
  }
  signature_name: "serving_default"
}
outputs {
  key: "dense_1"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 20
      }
    }
    float_val: 7.63123661e-11
    float_val: 4.39124115e-09
    float_val: 0.00193864899
    float_val: 2.18959837e-08
    float_val: 8.26274726e-12
    float_val: 7.78653364e-10
    float_val: 6.19444036e-12
    float_val: 0.000379952922
    float_val: 8.32395264e-11
    float_val: 0.0100553529
    float_val: 2.49862486e-09
    float_val: 1.52507184e-07
    float_val: 3.27335812e-07
    float_val: 1.79782049e-07
    float_val: 3.8991567e-07
    float_val: 3.28332533e-07
    float_val: 0.987624407
    float_val: 8.4118426e-08
    float_val: 1.53273311e-10
    float_val: 1.45717507e-07
  }
}

In [11]:
preds = pb_response.outputs["dense_1"].float_val

In [12]:
classes = [
    "Ragdoll",
    "Bombay",
    "American Curl",
    "Birman",
    "Egyptian Mau",
    "Bengal",
    "Siamese",
    "Maine Coon",
    "American Bobtail",
    "Turkish Angora",
    "Russian Blue",
    "British Shorthair",
    "Persian",
    "Abyssinian",
    "Norwegian Forest",
    "Manx",
    "Scottish Fold",
    "American Shorthair",
    "Exotic Shorthair",
    "Sphynx",
]

dict(zip(classes, preds))

{'Ragdoll': 7.63123661484677e-11,
 'Bombay': 4.39124114848255e-09,
 'American Curl': 0.0019386489875614643,
 'Birman': 2.18959836928434e-08,
 'Egyptian Mau': 8.26274725723719e-12,
 'Bengal': 7.786533640086191e-10,
 'Siamese': 6.1944403599578646e-12,
 'Maine Coon': 0.0003799529222305864,
 'American Bobtail': 8.323952638278342e-11,
 'Turkish Angora': 0.010055352933704853,
 'Russian Blue': 2.498624862568022e-09,
 'British Shorthair': 1.5250718377046724e-07,
 'Persian': 3.2733581178945315e-07,
 'Abyssinian': 1.79782048803645e-07,
 'Norwegian Forest': 3.899156695297279e-07,
 'Manx': 3.28332532717468e-07,
 'Scottish Fold': 0.9876244068145752,
 'American Shorthair': 8.411842600253294e-08,
 'Exotic Shorthair': 1.5327331071013361e-10,
 'Sphynx': 1.457175073937833e-07}